## **Process LC files in RA, Dec dirs**

In [22]:
import os
import glob
import tarfile
from urllib.request import urlretrieve
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import hstack



In [8]:
# Scipy
import scipy.stats as stats
from scipy.spatial import cKDTree as KDT

# Astropy
from astropy.stats import SigmaClip
from astropy.stats import sigma_clip, mad_std
from astropy.stats import sigma_clipped_stats

# astroML
from astroML.plotting import hist

In [225]:
def LCfilename(RA, Dec): 
        # given (RA, Dec), return RA/Dec directory names and LC file name 
        RAdir = getRAname(RA)
        DecDir = getDname(Dec)

        # for filename itself, use 6 decimal places for RA and Dec
        sRA = '%0.6f'%(RA)        
        sDec = '%0.6f'%(abs(Dec))
        sgnString = "p" 
        if Dec < 0:
           sgnString = "m"
        LCfilename = "LC_RA"+sRA+"_D"+sgnString+sDec+".dat"

        return RAdir, DecDir, LCfilename 

def getRAname(RA):         
        if RA > 180:
           RAname = "RAm" + str(int(abs(RA-360)))
        else:
           RAname = "RAp" + str(int(RA))
        return RAname

def getDname(Dec): 
        if Dec < 0:
           Dname = "Dm" + str(int(abs(10*Dec)))
        else:
           Dname = "Dp" + str(int(10*Dec))
        return Dname

In [17]:
cat_col_names = ['run1','col1','fld1','nctr','SSCra1','SSCdec1','SSCindx1',
      'distSS2WDasec','WDra','WDdec','delu','delg','delr','deli','delz',
      'WDMJD','WDobjc_type','WDobjc_rowc','WDobjc_colc','WDnchild',
      'WDrowc1','WDrowc2','WDrowc3','WDrowc4','WDrowc5',
      'WDrowcerr1','WDrowcerr2','WDrowcerr3','WDrowcerr4','WDrowcerr5',
      'WDcolc1','WDcolc2','WDcolc3','WDcolc4','WDcolc5',
      'WDcolcerr1','WDcolcerr2','WDcolcerr3','WDcolcerr4','WDcolcerr5',
      'WDpsfmag1','WDpsfmag2','WDpsfmag3','WDpsfmag4','WDpsfmag5',
      'WDpsfmagerr1','WDpsfmagerr2','WDpsfmagerr3','WDpsfmagerr4','WDpsfmagerr5',
      'WDmodelmag1','WDmodelmag2','WDmodelmag3','WDmodelmag4','WDmodelmag5',
      'WDmodelmagerr1','WDmodelmagerr2','WDmodelmagerr3','WDmodelmagerr4','WDmodelmagerr5',
      'umjd','gmjd','rmjd','imjd','zmjd',
      'umjdfrac','gmjdfrac','rmjdfrac','imjdfrac','zmjdfrac',
      'WDtai1','WDtai2','WDtai3','WDtai4','WDtai5',
      'WDtaifrac1','WDtaifrac2','WDtaifrac3','WDtaifrac4','WDtaifrac5',
      'WDairmass1','WDairmass2','WDairmass3','WDairmass4','WDairmass5',
      'WDpsf_fwhm1','WDpsf_fwhm2','WDpsf_fwhm3','WDpsf_fwhm4','WDpsf_fwhm5',
      'WDskyflux1','WDskyflux2','WDskyflux3','WDskyflux4','WDskyflux5',
      'objc_flags','objc_flags2',
      'flags_1','flags_2','flags_3','flags_4','flags_5',
      'flags2_1','flags2_2','flags2_3','flags2_4','flags2_5']


cat_col_dtype = {'run1':'int64','col1':'int64','fld1':'int64','nctr':'int64','SSCindx1':'int64',
                 'WDMJD':'int64','WDobjc_type':'int64','WDnchild':'int64',
                 'umjd':'int64','gmjd':'int64','rmjd':'int64','imjd':'int64','zmjd':'int64',
                'objc_flags':'int64','objc_flags2':'int64',
                'flags_1':'int64','flags_2':'int64','flags_3':'int64','flags_4':'int64','flags_5':'int64',
                 'flags2_1':'int64','flags2_2':'int64','flags2_3':'int64','flags2_4':'int64','flags2_5':'int64'}

# define col names for the SSC
Ncolnames = 'SSCindx,Nra,Ndec,NraRMS,NdecRMS,NnEpochs,NAR_val,Nu_Nobs,Nu_mMed,Nu_mMean,Nu_mErr,Nu_rms_scatt,Nu_chi2,Ng_Nobs,Ng_mMed,Ng_mMean,Ng_mErr,Ng_rms_scatt,Ng_chi2,Nr_Nobs,Nr_mMed,Nr_mMean,Nr_mErr,Nr_rms_scatt,Nr_chi2,Ni_Nobs,Ni_mMed,Ni_mMean,Ni_mErr,Ni_rms_scatt,Ni_chi2,Nz_Nobs,Nz_mMed,Nz_mMean,Nz_mErr,Nz_rms_scatt,Nz_chi2'
print('%s' % Ncolnames)

# Define the format string for the new SSC
newSSC_formstr = '%07d,%f,%f,%f,%f,%d,%f,%3d,%.4f,%.4f,%.4f,%.4f,%.4f,%3d,%.4f,%.4f,%.4f,%.4f,%.4f,%3d,%.4f,%.4f,%.4f,%.4f,%.4f,%3d,%.4f,%.4f,%.4f,%.4f,%.4f,%3d,%.4f,%.4f,%.4f,%.4f,%.4f\n'

SSCindx,Nra,Ndec,NraRMS,NdecRMS,NnEpochs,NAR_val,Nu_Nobs,Nu_mMed,Nu_mMean,Nu_mErr,Nu_rms_scatt,Nu_chi2,Ng_Nobs,Ng_mMed,Ng_mMean,Ng_mErr,Ng_rms_scatt,Ng_chi2,Nr_Nobs,Nr_mMed,Nr_mMean,Nr_mErr,Nr_rms_scatt,Nr_chi2,Ni_Nobs,Ni_mMed,Ni_mMean,Ni_mErr,Ni_rms_scatt,Ni_chi2,Nz_Nobs,Nz_mMed,Nz_mMean,Nz_mErr,Nz_rms_scatt,Nz_chi2


In [218]:
# robust standard deviation
def sigG(arr):
    return 0.741*(np.quantile(arr, 0.75)-np.quantile(arr, 0.25))

def LCtoMagStats(mag, magErr, NepochMin=2, dmMax=0.3, dsigMax=3.0, sysErr=0.002):
    # given an array of magnitudes mag and corresponding uncertainties magErr,
    # return quantities required for the standard star catalog: 
    #   'Nobs', 'mMed', 'mMean', 'mErr', 'rms_scatt', 'chi2'
    # where mMed and mMean are median and weighted mean value after the 
    # rejection of bad data points, Nobs is the number of surviving data points, 
    # rms_scatt is the robust standard deviation estimated from the 
    # interquartile range, mErr is the uncertainty of weighted median (for the
    # median, uncetainty is 1.25*rms_scatt/sqrt(Nobs)) and chi2 is the chi2
    # per degree of freedom: 1/(Nobs-1)*sum[(mag - mMean)^2/magErr^2], summed
    # over good points 
    
    # if the number of data points drops below NepochMin, adopt the following
    #   default values: Nobs=0, mMed = mMean = -9.99, mErr=rms_scatt=chi2=0  
    defaults = 0, -9.99, -9.99, 0, 0, 0 
    
    # processing steps: 
    # 0) add systematic photometric error, sysErr, in quadrature
    # 1) compute median and reject all points more than dmMax from the median
    # 2) recompute median and sigG and reject all points more than dsigMax*sigG
    #        from the median
    # 3) if at least NepochMin points survived, compute median and weighted mean
    # 
    # 0) and 1) 
    mag1 = mag[np.abs(mag-np.median(mag)) <= dmMax] 
    magErr1 = np.sqrt(magErr[np.abs(mag-np.median(mag)) <= dmMax]**2 + sysErr**2)
    if (np.size(mag1)<NepochMin):
        return defaults
    else: 
        # 2)
        sigG2 = sigG(mag1)
        mag2 = mag1[np.abs(mag1-np.median(mag1)) <= dsigMax*sigG2] 
        magErr2 = magErr1[np.abs(mag1-np.median(mag1)) <= dsigMax*sigG2] 
        if (np.size(mag2)<NepochMin):
            return defaults
    return magStats(mag2, magErr2)
 
def magStats(m, mErr):
    # given an array of magnitudes m and corresponding uncertainties mErr,
    # compute quantities required for the standard star catalog 
    Nobs = np.size(m)
    mMed = np.median(m)
    rms_scatt = sigG(m)
    # weighted quantities
    sumW = np.sum(1.0/mErr**2)
    mWmean = np.sum(m/mErr**2)/sumW
    WmeanErr = 1.0/np.sqrt(sumW)
    # robust chi2 per degree of freedom (that is, the width of chi distribution)
    chi = (m-mMed)/mErr
    chi2 = sigG(chi)
    return Nobs, mMed, mWmean, WmeanErr, rms_scatt, chi2

In [197]:
def getSSCentry(LC):
    entry = {}
    ra = np.median(LC['WDra'])
    dec = np.median(LC['WDdec'])
    raRMS = sigG(LC['WDra']) * 3600
    decRMS = sigG(LC['WDdec']) * 3600
    nEpochs = np.size(LC)
    AR_val = -9.99
    entry['coord'] = (ra, dec, raRMS, decRMS, nEpochs, AR_val)
    for b in ('1','2','3','4','5'):
        psf = LC['WDpsfmag'+b]
        psfErr = LC['WDpsfmagerr'+b]
        stats = LCtoMagStats(psf, psfErr)
        entry[b] = stats
        
    return entry    

### **DEFINE DIRS, CAT NAMES, ETC.**

In [178]:
def printStats(arr):
    print('           ', np.min(arr), np.mean(arr), np.median(arr), np.max(arr), sigG(arr), np.size(arr)) 
    return 

In [219]:
root_dir = '/Users/ivezic/Work/Science/CalibrationV2/LCs/LCdir'
LCfile = 'RAm007/Dm007/LC_RA352.602112_Dm0.789446.dat'
#LCfile = 'RAm009/Dp003/LC_RA350.914612_Dp0.312862.dat'
LCfilepath = root_dir + '/' +  LCfile
LCfilepath

'/Users/ivezic/Work/Science/CalibrationV2/LCs/LCdir/RAm007/Dm007/LC_RA352.602112_Dm0.789446.dat'

In [220]:
LC = Table.read(LCfilepath, format='ascii', names=cat_col_names)
SSCentry = getSSCentry(LC)

In [221]:
SSCentry

{'coord': (352.602143276,
  -0.7894517515,
  0.018521813737470437,
  0.02108871180019105,
  16,
  -9.99),
 '1': (4,
  24.174500000000002,
  24.185727638235637,
  0.5207801583264462,
  0.06287384999999958,
  0.05698298332936731),
 '2': (16,
  21.0596,
  21.056876737223828,
  0.010702508440497357,
  0.039606449999998516,
  0.9574345363508641),
 '3': (16,
  19.4896,
  19.48721289124762,
  0.005628106324186816,
  0.019228950000001254,
  0.8927974623911943),
 '4': (16,
  18.01325,
  18.013654699098137,
  0.004294083640756496,
  0.011633699999999213,
  0.7356993129217982),
 '5': (15,
  17.2095,
  17.20762566651848,
  0.00558005651858595,
  0.017487599999998733,
  0.7198963700420346)}

### looping...

In [295]:
%cd '/Users/ivezic/Work/Science/CalibrationV2/LCs/LCdir/RAm000/Dm000'
LClist = !ls LC*.dat
nLC = np.size(LClist)
nLC

/Users/ivezic/Work/Science/CalibrationV2/LCs/LCdir/RAm000/Dm000


191

In [313]:
workdir = !pwd 
SSCrow = ''
for i in range(0,nLC):
    LCfilepath = workdir[0] + '/' + LClist[i]
    LC = Table.read(LCfilepath, format='ascii', names=cat_col_names)
    SSCrow = getSSCentry(LC)
    ## write here to new SSC file (but note that extinction Ar is missing!)